In [2]:
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense,Concatenate, concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")
from valuelabs import AttentionLayer


Using TensorFlow backend.


The Dataset is as follows, I Created some features using SPACY-NER ,the statement was to predict 3 distractor for every Question and Answer statements

In [7]:
df_train = pd.read_csv('D:/Kaggle/Valulabs Hackerearth/DataSet/Train.csv',low_memory=False)

In [8]:
df_train.head()

,question,answer_text,distractor
0,Meals can be served,in rooms at 9:00 p. m.,"'outside the room at 3:00 p. m.', 'in the dini..."
1,It can be inferred from the passage that,The local government can deal with the problem...,"'If some tragedies occur again ', ' relevant d..."
2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,"'blame Tommy for his failing grades', 'blame T..."
3,It can be inferred from the passage that,the writer is not very willing to use idioms,'idioms are the most important part in a langu...
4,How can we deal with snake wounds according to...,Stay calm and do n't move .,'Cut the wound and suck the poison out .'


In [11]:
df_train_final = pd.read_csv('D:/Kaggle/Value_labs.csv')

In [12]:
df_train_final

,Unnamed: 0,question,answer_text,distractor,distractor_1,distractor_2,distractor_3,question_entity,question_label,question_label_exp,answer_entity,answer_label,answer_label_exp
0,0,Meals can be served,in rooms at 9:00 p. m.,"'outside the room at 3:00 p. m.', 'in the dini...",'outside the room at 3:00 p. m.','in the dining - room at 6:00 p. m.','in the dining - room from 7:30 a. m. to 9:15...,0,0,0,9:00 p. m.,TIME,Times smaller than a day
1,1,It can be inferred from the passage that,The local government can deal with the problem...,"'If some tragedies occur again ', ' relevant d...",'If some tragedies occur again ',' relevant departments of the State Council s...,'Currently ',0,0,0,0,0,0
2,2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,"'blame Tommy for his failing grades', 'blame T...",'blame Tommy for his failing grades','blame Tommy for his failing grades','blame Tommy for his failing grades',Tommy,ORG,"Companies, agencies, institutions, etc.",0,0,0
3,3,It can be inferred from the passage that,the writer is not very willing to use idioms,'idioms are the most important part in a langu...,'idioms are the most important part in a langu...,'nonnative speakers should learn more idioms','there are no ways to master idioms',0,0,0,0,0,0
4,4,How can we deal with snake wounds according to...,Stay calm and do n't move .,'Cut the wound and suck the poison out .','Cut the wound and suck the poison out .','Cut the wound and suck the poison out .','Cut the wound and suck the poison out .',0,0,0,0,0,0
5,5,What was the writer 's problem when she studie...,She missed her family very much .,"""She did n't like her new school ."", ""She did ...","""She did n't like her new school .""","""She did n't get along well with her classmat...","""She did n't like her new teacher .""",0,0,0,0,0,0
6,6,Who were killed on February 5 in a small town ...,Chen Jianqing and one of her partners,"'Chen Jianqing and her husband', 'Chen Jingmin...",'Chen Jianqing and her husband','Chen Jingmin and a Hong Kong businessman','Chen Jingmin and one of his s',February 5,DATE,Absolute or relative dates or periods,Chen Jianqing,PERSON,"People, including fictional"
7,7,"According to the writer , which of the followi...","Soccer is popular all over the world , but tru...",'Millions of people all over the world are pla...,'Millions of people all over the world are pla...,' but only seven countries have ever had famo...,'Only seven countries from South America and ...,0,0,0,0,0,0
8,8,During a fire children often,panic,'know certain steps','know certain steps','know certain steps','know certain steps',0,0,0,0,0,0
9,9,What 's the title of the passage ?,Five children died in a kindergarten bus accid...,"'A bus accident in Deng zhou .', 'All primary ...",'A bus accident in Deng zhou .','All primary school and kindergarten vehicles...,'The driver of the bus has been put into cust...,0,0,0,Five,CARDINAL,Numerals that do not fall under another type


Some DataCleaning

In [4]:
negations_dic = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [5]:
stop_words = set(stopwords.words('english')) 
def question_cleaner(text):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
#     newString = ' '.join([negations_dic[t] if t in negations_dic else t for t in newString.split(" ")])    
#     newString = re.sub(r"'s\b","",newString)
#     newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]
#     long_words=[]
    return (" ".join(tokens)).strip()

def answer_cleaner(text):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub("'",'', newString)
#     newString = ' '.join([negations_dic[t] if t in negations_dic else t for t in newString.split(" ")])    
#     newString = re.sub(r"'s\b","",newString)
#     newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]   
    return (" ".join(tokens)).strip()

def distractor_cleaner(text):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub("'",'', newString)
#     newString = ' '.join([negations_dic[t] if t in negations_dic else t for t in newString.split(" ")])    
#     newString = re.sub(r"'s\b","",newString)
#     newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]   
    return (" ".join(tokens)).strip()

In [6]:
df_train

,Unnamed: 0,question,answer_text,distractor,distractor_1,distractor_2,distractor_3,question_entity,question_label,question_label_exp,answer_entity,answer_label,answer_label_exp
0,0,Meals can be served,in rooms at 9:00 p. m.,"'outside the room at 3:00 p. m.', 'in the dini...",'outside the room at 3:00 p. m.','in the dining - room at 6:00 p. m.','in the dining - room from 7:30 a. m. to 9:15...,0,0,0,9:00 p. m.,TIME,Times smaller than a day
1,1,It can be inferred from the passage that,The local government can deal with the problem...,"'If some tragedies occur again ', ' relevant d...",'If some tragedies occur again ',' relevant departments of the State Council s...,'Currently ',0,0,0,0,0,0
2,2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,"'blame Tommy for his failing grades', 'blame T...",'blame Tommy for his failing grades','blame Tommy for his failing grades','blame Tommy for his failing grades',Tommy,ORG,"Companies, agencies, institutions, etc.",0,0,0
3,3,It can be inferred from the passage that,the writer is not very willing to use idioms,'idioms are the most important part in a langu...,'idioms are the most important part in a langu...,'nonnative speakers should learn more idioms','there are no ways to master idioms',0,0,0,0,0,0
4,4,How can we deal with snake wounds according to...,Stay calm and do n't move .,'Cut the wound and suck the poison out .','Cut the wound and suck the poison out .','Cut the wound and suck the poison out .','Cut the wound and suck the poison out .',0,0,0,0,0,0
5,5,What was the writer 's problem when she studie...,She missed her family very much .,"""She did n't like her new school ."", ""She did ...","""She did n't like her new school .""","""She did n't get along well with her classmat...","""She did n't like her new teacher .""",0,0,0,0,0,0
6,6,Who were killed on February 5 in a small town ...,Chen Jianqing and one of her partners,"'Chen Jianqing and her husband', 'Chen Jingmin...",'Chen Jianqing and her husband','Chen Jingmin and a Hong Kong businessman','Chen Jingmin and one of his s',February 5,DATE,Absolute or relative dates or periods,Chen Jianqing,PERSON,"People, including fictional"
7,7,"According to the writer , which of the followi...","Soccer is popular all over the world , but tru...",'Millions of people all over the world are pla...,'Millions of people all over the world are pla...,' but only seven countries have ever had famo...,'Only seven countries from South America and ...,0,0,0,0,0,0
8,8,During a fire children often,panic,'know certain steps','know certain steps','know certain steps','know certain steps',0,0,0,0,0,0
9,9,What 's the title of the passage ?,Five children died in a kindergarten bus accid...,"'A bus accident in Deng zhou .', 'All primary ...",'A bus accident in Deng zhou .','All primary school and kindergarten vehicles...,'The driver of the bus has been put into cust...,0,0,0,Five,CARDINAL,Numerals that do not fall under another type


In [7]:
question_clean = []
for t in df_train['question']:
    question_clean.append(question_cleaner(t))
answer_clean = []
for t in df_train['answer_text']:
    answer_clean.append(answer_cleaner(t))
distractor_clean=[]
for t in df_train['distractor_1']:
    distractor_clean.append(distractor_cleaner(t))	
df_train['question_clean'] = question_clean
df_train['answer_clean'] = answer_clean
df_train['distractor_clean']=distractor_clean
df_train.drop(['question','answer_text','distractor','distractor_1','distractor_2',
               'distractor_3','question_entity','question_label','answer_entity',
               'answer_label','question_label_exp','answer_label_exp'],axis=1,inplace=True)
df_train.drop('Unnamed: 0',axis=1,inplace=True)

In [8]:
df_train['distractor_clean'] = df_train['distractor_clean'].apply(lambda x:'start '+x+' end' ) 

In [9]:
df_train.isnull().sum()

question_clean      0
answer_clean        0
distractor_clean    0
dtype: int64

In [10]:
y = df_train['distractor_clean']

In [11]:
df_train.drop('distractor_clean',inplace=True,axis=1)

In [12]:
# df_train.drop('question_clean',inplace=True,axis=1)

we checked the length of the question, answers and each distractor by separating all the distractors ,to feed to LSTM

In [13]:
max_question_len = 9
max_answer_len = 9
max_distractor_len = 7


Now I m using Seq2seq model using Keras with LSTMs Now Lstms are not very good at predicting long sentences but lucky for us
we dont have long sentences so we also use an attention layer for better predictions its a custom attention layer
keras doesnt have a support for that , this layer was taken from a AnalyticsVidhya

In [14]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(df_train,y,test_size=0.1,random_state=0,shuffle=True) 

In [15]:
x_tr.shape

(28349, 2)

In [16]:
quest_tr = x_tr['question_clean']
quest_val = x_val['question_clean']

ans_tr = x_tr['answer_clean'] 
ans_val =x_val['answer_clean']

In [17]:
x_tokenizer = Tokenizer(num_words=4526)
x_tokenizer.fit_on_texts(list(quest_tr))

In [18]:
quest_tr_seq    =   x_tokenizer.texts_to_sequences(quest_tr)
quest_val_seq   =   x_tokenizer.texts_to_sequences(quest_val)
#padding zero upto maximum length
quest_tr    =   pad_sequences(quest_tr_seq,  maxlen=max_question_len, padding='post') 
quest_val  =   pad_sequences(quest_val_seq, maxlen=max_question_len, padding='post')
x_voc_size   =  x_tokenizer.num_words +1

In [19]:
x_tr.shape

(28349, 2)

In [20]:
y_tokenizer = Tokenizer(num_words=4526)
y_tokenizer.fit_on_texts(list(ans_tr))

#convert text sequences into integer sequencesans
ans_tr_seq =   y_tokenizer.texts_to_sequences(ans_tr) 
ans_val_seq   =   y_tokenizer.texts_to_sequences(ans_val)

# padding zero upto maximum length
ans_tr    =   pad_sequences(ans_tr_seq,  maxlen=max_answer_len, padding='post') 
ans_val  =   pad_sequences(ans_val_seq, maxlen=max_answer_len, padding='post')
y_voc_size   =  y_tokenizer.num_words +1

In [21]:
ans_tr.shape

(28349, 9)

In [22]:
z_tokenizer = Tokenizer(num_words=3280)
z_tokenizer.fit_on_texts(list(y_tr))

#convert summary sequences into integer sequences
y_tr_seq   =   z_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   z_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_distractor_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_distractor_len, padding='post')


In [23]:
z_voc_size  =  z_tokenizer.num_words+1

In [24]:
y_tr.shape

(28349, 7)

In [25]:

ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr,ind, axis=0)
quest_tr=np.delete(quest_tr,ind, axis=0)
ans_tr=np.delete(ans_tr,ind, axis=0)

ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)
y_val=np.delete(y_val,ind, axis=0)
quest_val=np.delete(quest_val,ind, axis=0)
ans_val=np.delete(ans_val,ind, axis=0)    

In [26]:
latent_dim = 300

# Encoder 
encoder_inputs = Input(shape=(max_question_len,)) 
enc_emb = Embedding(x_voc_size,100,trainable=True)(encoder_inputs) 

#LSTM 1 
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4) 
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4) 
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

encoder_lstm3 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4) 
encoder_outputs, state_h3, state_c3 = encoder_lstm3(encoder_output2)

#  Encoder 
encoder1_inputs = Input(shape=(max_answer_len,)) 
enc1_emb = Embedding(y_voc_size, 100,trainable=True)(encoder1_inputs) 

# LSTM 1 
encoder1_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4) 
encoder1_output1, state1_h1, state1_c1 = encoder1_lstm1(enc1_emb) 

encoder1_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4) 
encoder1_output2, state1_h2, state1_c2 = encoder1_lstm2(encoder1_output1)

encoder1_lstm3 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4) 
encoder1_outputs, state1_h3, state1_c3 = encoder1_lstm3(encoder1_output2)

W1013 17:44:58.859004  9344 deprecation.py:506] From D:\Anaconda3\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1013 17:44:58.911041  9344 deprecation.py:506] From D:\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [27]:
merged_output = concatenate([encoder_outputs,encoder1_outputs])
encoder2_lstm1=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4) 
encoder2_output1, state2_h1, state2_c1= encoder2_lstm1(merged_output)
# encoder2_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4) 
# encoder2_output2, state2_h2, state2_c2 = encoder2_lstm2(encoder2_output1) 
# encoder2_lstm3 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4) 
# encoder2_output3, state2_h3, state2_c3 = encoder2_lstm2(encoder2_output2) 

In [ ]:
merged_output.shape

In [29]:
decoder_inputs = Input(shape=(None,)) 
dec_emb_layer = Embedding(z_voc_size,100) 
dec_emb = dec_emb_layer(decoder_inputs) 

decoder_lstm = LSTM(latent_dim, return_sequences=True) 
decoder_outputs= decoder_lstm(dec_emb) 

In [30]:
attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([merged_output, decoder_outputs])

decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#Dense layer
decoder_dense = TimeDistributed(Dense(z_voc_size, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_concat_input)

model = Model([encoder_inputs,encoder1_inputs, decoder_inputs], decoder_outputs) 
model.summary()

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 9)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 9)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 9, 100)       452700      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 9, 100)       452700      input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
# encoder_inputs = Input(shape=(max_question_len,)) 
# enc_emb = Embedding(4000,100,trainable=True)(encoder_inputs) 

In [ ]:
# op = Merged([enc_emb1,enc_emb])

In [ ]:
reverse_target_word_index[0]

In [ ]:
x_tr.shape

In [ ]:
ilename = 'model.h1.dist3'
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)
model.fit([ans_tr ,quest_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=50,callbacks=[es],batch_size=128, validation_data=([ans_val,quest_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

reverse_target_word_index=z_tokenizer.index_word 
reverse_source_word_index_ans=y_tokenizer.index_word
reverse_source_word_index_ques=x_tokenizer.index_word 
target_word_index=z_tokenizer.word_index

Train on 27996 samples, validate on 3091 samples


W1013 17:45:58.033468  9344 deprecation.py:323] From D:\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
27996/27996 [==============================] - 1192s 43ms/sample - loss: 4.7055 - val_loss: 4.3782
Epoch 2/50
 5888/27996 [=====>........................] - ETA: 13:56 - loss: 4.4275

Here we create model Inference to predict output as text

In [ ]:
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder2_output1, state2_h1, state2_c1])

In [ ]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_answer_len,400))


In [ ]:
dec_emb2= dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat)

# Final decoder model
decoder_model = Model(
[decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
[decoder_outputs2] + [state_h2, state_c2])

In [ ]:
def decode_sequence(input_seq,inputs_seq1):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq,input_seq1)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))

    # Chose the 'start' word as the first word of the target sequence
    target_seq[0, 0] = target_word_index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if sampled_token_index == 0:
            sampled_token = reverse_target_word_index[sampled_token_index+1]
        else:
            sampled_token=reverse_target_word_index[sampled_token_index]

        if(sampled_token!='end'):
            decoded_sentence += ' '+sampled_token

            # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'end'  or len(decoded_sentence.split()) >= (max_distractor_len-1)):
            stop_condition = True


        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
      if((i!=0 and i!=target_word_index['start']) and i!=target_word_index['end']):
        newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2ques(input_seq):
    newString=''
    for i in input_seq:
      if(i!=0):
        newString=newString+reverse_source_word_index_ques[i]+' '
    return newString

def seq2ans(input_seq):
    newString=''
    for i in input_seq:
      if(i!=0):
        newString=newString+reverse_source_word_index_ans[i]+' '
    return newString

for i in range(len(x_val)):
#   print("Question",seq2ques(quest_val[i]))
  print("Answer",seq2ans(ans_val[i]))
  print("Original distractor:",seq2summary(y_val[i]))
  print("Predicted distactor:",decode_sequence(ans_val[i].reshape(1,max_answer_len)))
  print("\n")


In [ ]:
df_test = pd.read_csv('D:/Kaggle/Valulabs/DataSet/Test.csv',low_memory=False)

In [ ]:
df_test.drop('question',axis=1,inplace=True)

In [ ]:
distractor_clean=[]
for t in df_test['answer_text']:
    distractor_clean.append(distractor_cleaner(t))

In [ ]:
df_test['answer_clean'] = distractor_clean

In [ ]:
ans_test = df_test['answer_clean']

In [ ]:
ans_test_seq   =   y_tokenizer.texts_to_sequences(ans_test.values)
ans_test   =   pad_sequences(ans_test_seq, maxlen=max_answer_len, padding='post')

In [ ]:
ans_test.shape

In [ ]:
preds = []
counter=0
for i in range(len(ans_test)):
    k = decode_sequence(ans_test[i].reshape(1,max_answer_len))
    counter=counter+1
    preds.append(k)
    print(counter)

In [ ]:
dic={
    'distractor_1':preds
}
df_final = pd.DataFrame(data=dic)
df_final.to_csv('first_dis.csv')


In [ ]:
df_test = pd.read_csv('D:/Kaggle/Valulabs/DataSet/Results.csv',low_memory=False)
df_dis1 = pd.read_csv('D:/Kaggle/first_dis.csv',low_memory=False)
df_dis2 = pd.read_csv('D:/Kaggle/second_dis.csv',low_memory=False)
df_dis3 = pd.read_csv('D:/Kaggle/third_dis.csv',low_memory=False)

In [ ]:
df_dis3

In [ ]:
df_test['distractor'] = "'"+ df_dis1['distractor_1']+"'"+","+"'"+ df_dis2['distractor_2']+"'"+","+"'"+ df_dis3['distractor_3']+"'"

In [ ]:
df_test.to_csv('Val_sub.csv',index=False)

This Competition one of the most difficult ones as i still am learning about the Neural Networks but it was interesting, the state of the art achieves about
30% Accuracy for predicting distractors I got  about 6% which is not good but with more learning i could do better
This Competition has no leaderboard